In [1]:
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.cuda.amp import GradScaler, autocast
import torch.nn.functional as F
from torch.nn import Conv2d, BatchNorm2d, ReLU, Sigmoid

from torchvision import models


if torch.cuda.is_available():
    print("CUDA is available. PyTorch is running on the GPU.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. PyTorch is running on the CPU.")

CUDA is available. PyTorch is running on the GPU.
Device name: Quadro RTX 8000


# Import Data

In [3]:
target_layer = '09'

data_dir = '/vast/xj2173/diffeo/data/temp_inv_data/'
numbers = [f"{i:02}" for i in range(40)]
data_name = [data_dir + f'15-100-4-4-3-224-224_image-00{i}_activation_layer-{target_layer}.pt' for i in numbers]

target_layer = int(target_layer)

In [4]:
# data[0] is the 0th picture, data[1] is the 1st picture, etc..
data = [torch.load(file_name, map_location='cpu') for file_name in tqdm(data_name)]
data = torch.stack(data, dim=0)
data.shape

100%|██████████| 40/40 [01:10<00:00,  1.77s/it]


torch.Size([40, 1500, 48, 56, 56])

- 40 different images
- 1500 different diffeos
    - 15 diffeo strengths
    - 100 individual diffeos per diffeo strength
- 48 channels
- 56 x 56 image

In [11]:
target_channel = 1
num_of_features_cutoff = 1500

features = data[:, 1:, target_channel, :, :]
pics, diffeo, size, _ = features.shape
features = features.permute(1, 0, 2, 3)
features = features.reshape(pics * diffeo, size, size)
features = features[:num_of_features_cutoff, :, :]

class NaiveInverseDiffeo(Dataset):
    def __init__(self, transform=None):
        self.labels = data[:, 0, target_channel, :, :]
        self.num_pics = pics
        
    def __len__(self):
        return int(features.shape[0])

    def __getitem__(self, idx):
        num_pics = self.num_pics
        label = self.labels[idx % num_pics]
        feature = features[idx]
        
        return feature, label


dataset = NaiveInverseDiffeo()

In [12]:
features[0].shape

torch.Size([56, 56])

In [13]:
dataset.labels[0].shape

torch.Size([56, 56])

# Objective

We've experimentally confirmed:
$$
g_{naive}^{-1} \star N_i (g \cdot I) = N_i(I) + \eta_i
$$
Where $g_{naive}^{-1}$ is just the inverse of $g$ using the $\cdot$ algebra. So basically, it looks like the original but w/ some noise $\eta_i$. 

Now the objective is to see if we can learn $\eta_i$. Or rewritten in another way
$$
h^{-1} \star g_{naive}^{-1} \star  N_i (g \cdot I) = N_i(I)
$$
We want to learn $h^{-1}$, s.t. the 2nd equation is true


# Learning $h^{-1}$

- 40 different images
- 15 diffeo strengths
- 100 individual diffeos per diffeo strength
- 48 channels
- 56 x 56 image

### Split Data

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:
split = 0.8
batch_size = 25

train_size = int(split * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(dataset=val_dataset,   batch_size=batch_size, shuffle=True)

### Initalize Model

In [25]:
# Load the pretrained EfficientNetB0 model
model = models.efficientnet_b0(pretrained=True)

## Get Nodes
def flatten(array):
    result = []
    for element in array:
        if hasattr(element, "__iter__"):
            result.extend(flatten(element))
        else:
            result.append(element)
    return result

layers = list(model.children())
layers_flat = flatten(layers)[:]

In [28]:
print(layers_flat[: target_layer + 1])

[Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), SiLU(inplace=True), MBConv(
  (block): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): SqueezeExcitation(
      (avgpool): AdaptiveAvgPool2d(output_size=1)
      (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
      (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
      (activation): SiLU(inplace=True)
      (scale_activation): Sigmoid()
    )
    (2): Conv2dNormActivation(
      (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (stochastic_depth): StochasticDepth(p=0.0

In [29]:
# Create a new model that ends at layer 13
model_trunc = torch.nn.Sequential(*layers_flat[: target_layer + 1])